# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import FeatureUnion, Pipeline 

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
customers.shape

(191652, 369)

In [4]:
customers.to_csv('customers.csv')

In [5]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
azdias.shape

(891221, 366)

In [ ]:
azdias.to_csv('azdias.csv')

In [ ]:
for var in customers.columns:
    print(var)

###  Define variable's data type 

In [ ]:
var_numeric = ['ANZ_HAUSHALTE_AKTIV','ANZ_HH_TITEL','ANZ_KINDER','ANZ_PERSONEN','ANZ_STATISTISCHE_HAUSHALTE','ANZ_TITEL','KBA13_ANZAHL_PKW']

var_ordinal = ['AKT_DAT_KL','ALTER_HH','ALTER_KIND1','ALTER_KIND2','ALTER_KIND3','ALTER_KIND4','ARBEIT','BALLRAUM','D19_BANKEN_ANZ_12','D19_BANKEN_ANZ_24','D19_BANKEN_DATUM','D19_BANKEN_OFFLINE_DATUM','D19_BANKEN_ONLINE_DATUM','D19_BANKEN_ONLINE_QUOTE_12','D19_GESAMT_ANZ_12','D19_GESAMT_ANZ_24','D19_GESAMT_DATUM','D19_GESAMT_OFFLINE_DATUM','D19_GESAMT_ONLINE_DATUM','D19_GESAMT_ONLINE_QUOTE_12','D19_TELKO_ANZ_12','D19_TELKO_ANZ_24','D19_TELKO_DATUM','D19_TELKO_OFFLINE_DATUM','D19_TELKO_ONLINE_DATUM','D19_TELKO_ONLINE_QUOTE_12','D19_VERSAND_ANZ_12','D19_VERSAND_ANZ_24','D19_VERSAND_DATUM','D19_VERSAND_OFFLINE_DATUM','D19_VERSAND_ONLINE_DATUM','D19_VERSAND_ONLINE_QUOTE_12','D19_VERSI_ANZ_12','D19_VERSI_ANZ_24','D19_VERSI_DATUM','D19_VERSI_OFFLINE_DATUM','D19_VERSI_ONLINE_DATUM','D19_VERSI_ONLINE_QUOTE_12','EWDICHTE','FINANZ_ANLEGER','FINANZ_HAUSBAUER','FINANZ_MINIMALIST','FINANZ_SPARER','FINANZ_UNAUFFAELLIGER','FINANZ_VORSORGER','FIRMENDICHTE','HH_EINKOMMEN_SCORE','INNENSTADT','KBA05_ALTER1','KBA05_ALTER2','KBA05_ALTER3','KBA05_ALTER4','KBA05_ANHANG','KBA05_ANTG1','KBA05_ANTG2','KBA05_ANTG3','KBA05_ANTG4','KBA05_AUTOQUOT','KBA05_BAUMAX','KBA05_CCM1','KBA05_CCM2','KBA05_CCM3','KBA05_CCM4','KBA05_DIESEL','KBA05_FRAU','KBA05_GBZ','KBA05_HERST1','KBA05_HERST2','KBA05_HERST3','KBA05_HERST4','KBA05_HERST5','KBA05_KRSAQUOT','KBA05_KRSHERST1','KBA05_KRSHERST2','KBA05_KRSHERST3','KBA05_KRSKLEIN','KBA05_KRSOBER','KBA05_KRSVAN','KBA05_KRSZUL','KBA05_KW1','KBA05_KW2','KBA05_KW3','KBA05_MAXAH','KBA05_MAXBJ','KBA05_MAXSEG','KBA05_MAXVORB','KBA05_MOD1','KBA05_MOD2','KBA05_MOD3','KBA05_MOD4','KBA05_MOD8','KBA05_MODTEMP','KBA05_MOTOR','KBA05_MOTRAD','KBA05_SEG1','KBA05_SEG10','KBA05_SEG2','KBA05_SEG3','KBA05_SEG4','KBA05_SEG5','KBA05_SEG7','KBA05_SEG8','KBA05_SEG9','KBA05_VORB0','KBA05_VORB1','KBA05_VORB2','KBA05_ZUL1','KBA05_ZUL2','KBA05_ZUL3','KBA05_ZUL4','KBA13_ALTERHALTER_30','KBA13_ALTERHALTER_45','KBA13_ALTERHALTER_60','KBA13_ALTERHALTER_61','KBA13_ANTG1','KBA13_ANTG2','KBA13_ANTG3','KBA13_ANTG4','KBA13_AUDI','KBA13_AUTOQUOTE','KBA13_BAUMAX','KBA13_BJ_1999','KBA13_BJ_2000','KBA13_BJ_2004','KBA13_BJ_2006','KBA13_BJ_2008','KBA13_BJ_2009','KBA13_BMW','KBA13_CCM_0_1400','KBA13_CCM_1000','KBA13_CCM_1200','KBA13_CCM_1400','KBA13_CCM_1401_2500','KBA13_CCM_1500','KBA13_CCM_1600','KBA13_CCM_1800','KBA13_CCM_2000','KBA13_CCM_2500','KBA13_CCM_2501','KBA13_CCM_3000','KBA13_CCM_3001','KBA13_FAB_ASIEN','KBA13_FAB_SONSTIGE','KBA13_FIAT','KBA13_FORD','KBA13_GBZ','KBA13_HALTER_20','KBA13_HALTER_25','KBA13_HALTER_30','KBA13_HALTER_35','KBA13_HALTER_40','KBA13_HALTER_45','KBA13_HALTER_50','KBA13_HALTER_55','KBA13_HALTER_60','KBA13_HALTER_65','KBA13_HALTER_66','KBA13_HERST_ASIEN','KBA13_HERST_AUDI_VW','KBA13_HERST_BMW_BENZ','KBA13_HERST_EUROPA','KBA13_HERST_FORD_OPEL','KBA13_HERST_SONST','KBA13_HHZ','KBA13_KMH_0_140','KBA13_KMH_110','KBA13_KMH_140','KBA13_KMH_140_210','KBA13_KMH_180','KBA13_KMH_210','KBA13_KMH_211','KBA13_KMH_250','KBA13_KMH_251','KBA13_KRSAQUOT','KBA13_KRSHERST_AUDI_VW','KBA13_KRSHERST_BMW_BENZ','KBA13_KRSHERST_FORD_OPEL','KBA13_KRSSEG_KLEIN','KBA13_KRSSEG_OBER','KBA13_KRSSEG_VAN','KBA13_KRSZUL_NEU','KBA13_KW_0_60','KBA13_KW_110','KBA13_KW_120','KBA13_KW_121','KBA13_KW_30','KBA13_KW_40','KBA13_KW_50','KBA13_KW_60','KBA13_KW_61_120','KBA13_KW_70','KBA13_KW_80','KBA13_KW_90','KBA13_MAZDA','KBA13_MERCEDES','KBA13_MOTOR','KBA13_NISSAN','KBA13_OPEL','KBA13_PEUGEOT','KBA13_RENAULT','KBA13_SEG_GELAENDEWAGEN','KBA13_SEG_GROSSRAUMVANS','KBA13_SEG_KLEINST','KBA13_SEG_KLEINWAGEN','KBA13_SEG_KOMPAKTKLASSE','KBA13_SEG_MINIVANS','KBA13_SEG_MINIWAGEN','KBA13_SEG_MITTELKLASSE','KBA13_SEG_OBEREMITTELKLASSE','KBA13_SEG_OBERKLASSE','KBA13_SEG_SONSTIGE','KBA13_SEG_SPORTWAGEN','KBA13_SEG_UTILITIES','KBA13_SEG_VAN','KBA13_SEG_WOHNMOBILE','KBA13_SITZE_4','KBA13_SITZE_5','KBA13_SITZE_6','KBA13_TOYOTA','KBA13_VORB_0','KBA13_VORB_1','KBA13_VORB_1_2','KBA13_VORB_2','KBA13_VORB_3','KBA13_VW','KKK','KOMBIALTER','MOBI_RASTER','MOBI_REGIO','ORTSGR_KLS9','PLZ8_ANTG1','PLZ8_ANTG2','PLZ8_ANTG3','PLZ8_ANTG4','PLZ8_BAUMAX','PLZ8_GBZ','PLZ8_HHZ','RELAT_AB','RT_KEIN_ANREIZ','RT_SCHNAEPPCHEN','RT_UEBERGROESSE','SEMIO_DOM','SEMIO_ERL','SEMIO_FAM','SEMIO_KAEM','SEMIO_KRIT','SEMIO_KULT','SEMIO_LUST','SEMIO_MAT','SEMIO_PFLICHT','SEMIO_RAT','SEMIO_REL','SEMIO_SOZ','SEMIO_TRADV','SEMIO_VERT','STRUKTURTYP','UMFELD_ALT','UMFELD_JUNG','VHA','VHN','VK_DHT4A','VK_DISTANZ','VK_ZG11','W_KEIT_KIND_HH','WOHNDAUER_2008','ALTERSKATEGORIE_GROB']

var_binary = ['GREEN_AVANTGARDE','HH_DELTA_FLAG','KBA05_SEG6','KONSUMZELLE','SOHO_KZ','UNGLEICHENN_FLAG','ANREDE_KZ']

var_date = ['EINGEFUEGT_AM','EINGEZOGENAM_HH_JAHR','GEBURTSJAHR','MIN_GEBAEUDEJAHR']

var_categorical = ['AGER_TYP','ALTERSKATEGORIE_FEIN','CAMEO_DEU_2015','CAMEO_DEUG_2015','CAMEO_INTL_2015','CJT_GESAMTTYP','CJT_KATALOGNUTZER','CJT_TYP_1','CJT_TYP_2','CJT_TYP_3','CJT_TYP_4','CJT_TYP_5','CJT_TYP_6','D19_BANKEN_DIREKT','D19_BANKEN_GROSS','D19_BANKEN_LOKAL','D19_BANKEN_REST','D19_BEKLEIDUNG_GEH','D19_BEKLEIDUNG_REST','D19_BILDUNG','D19_BIO_OEKO','D19_BUCH_CD','D19_DIGIT_SERV','D19_DROGERIEARTIKEL','D19_ENERGIE','D19_FREIZEIT','D19_GARTEN','D19_HANDWERK','D19_HAUS_DEKO','D19_KINDERARTIKEL','D19_KONSUMTYP','D19_KONSUMTYP_MAX','D19_KOSMETIK','D19_LEBENSMITTEL','D19_LETZTER_KAUF_BRANCHE','D19_LOTTO','D19_NAHRUNGSERGAENZUNG','D19_RATGEBER','D19_REISEN','D19_SAMMELARTIKEL','D19_SCHUHE','D19_SONSTIGE','D19_SOZIALES','D19_TECHNIK','D19_TELKO_MOBILE','D19_TELKO_REST','D19_TIERARTIKEL','D19_VERSAND_REST','D19_VERSICHERUNGEN','D19_VOLLSORTIMENT','D19_WEIN_FEINKOST','DSL_FLAG','EXTSEL992','FINANZTYP','GEBAEUDETYP','GEBAEUDETYP_RASTER','GEMEINDETYP','GFK_URLAUBERTYP','HEALTH_TYP','KBA05_HERSTTEMP','KBA05_MAXHERST','KK_KUNDENTYP','KONSUMNAEHE','LP_FAMILIE_FEIN','LP_FAMILIE_GROB','LP_LEBENSPHASE_FEIN','LP_LEBENSPHASE_GROB','LP_STATUS_FEIN','LP_STATUS_GROB','NATIONALITAET_KZ','ONLINE_AFFINITAET','OST_WEST_KZ','PRAEGENDE_JUGENDJAHRE','REGIOTYP','RETOURTYP_BK_S','SHOPPER_TYP','TITEL_KZ','VERDICHTUNGSRAUM','VERS_TYP','WOHNLAGE','ZABEOTYP']

var_other = ['CUSTOMER_GROUP', 'LNR', 'ONLINE_PURCHASE', 'PRODUCT_GROUP']

In [ ]:
#check there is no variable missing out.
print(len(customers.columns) == len(var_numeric+var_ordinal+var_binary+var_categorical+var_date+var_other))
print(set(customers.columns) == set(var_numeric+var_ordinal+var_binary+var_categorical+var_date+var_other))

In [ ]:
for var in customers.columns:
    if customers[var].dtype == 'object' and var not in var_categorical:
        print(var, customers[var].dtype)
        
# EINGEFUEGT_AM -> var_date
# PRODUCT_GROUP -> var_other
# CUSTOMER_GROUP -> var_other

In [ ]:
for var in azdias.columns:
    if azdias[var].dtype == 'object' and var not in var_categorical:
        print(var, azdias[var].dtype)

#### Data Preprocessing for numeric variables

In [ ]:
for var in var_numeric:
    print(var, customers[var].nunique())

In [ ]:
for var in var_numeric:
    print(var, customers[var].unique()[0:20])

In [ ]:
customers[var_numeric].describe()

In [ ]:
azdias[var_numeric].describe()

In [ ]:
# check two things: dtype and negative value
def CheckNumeric(data):
    var_numeric =  ['ANZ_HAUSHALTE_AKTIV','ANZ_HH_TITEL','ANZ_KINDER','ANZ_PERSONEN','ANZ_STATISTISCHE_HAUSHALTE','ANZ_TITEL','KBA13_ANZAHL_PKW']
    cnt = 0
    for var in var_numeric:
        if customers[var].dtype == 'object':
            print(var, 'is not numeric variable')
            cnt += 1
        if customers[var].min() < 0:
            print(var, 'has some abnormal values!')
            cnt += 1
    if cnt == 0:
        print('All checked!')
    cnt = 0

In [ ]:
CheckNumeric(customers)

In [ ]:
CheckNumeric(azdias)

#### Data Preprocessing for binary variables

In [ ]:
for var in var_binary:
    print(var, customers[var].unique())

In [ ]:
for var in var_binary:
    print(var, azdias[var].unique())

In [ ]:
# recode [1, 2] into [0,1] and recode all values other than [0,1] into missing value

def RecoderBinary(data):
    var_binary = ['GREEN_AVANTGARDE','HH_DELTA_FLAG','KBA05_SEG6','KONSUMZELLE','SOHO_KZ','UNGLEICHENN_FLAG','ANREDE_KZ','ANREDE_KZ']
    data.loc[:,'ANREDE_KZ'] = data['ANREDE_KZ'].map({1:0,2:1})
    
    for var in var_binary:
        l = data[var].unique()
        for e in l[~(np.isnan(l))]:
            if e not in [0,1]:
                print(var, 'has been recoded.')
                data.loc[:,var] = data[var].map({0:0,1:1})           
            
    print('All checked!')

In [ ]:
RecoderBinary(customers)

In [ ]:
RecoderBinary(azdias)

#### Data Preprocessing for ordinal variables

In [ ]:
for var in var_ordinal:
    if customers[var].nunique() > 10:
        print(var, customers[var].nunique())

In [ ]:
for var in var_ordinal:
    print(var, sorted(customers[var].unique()))

In [ ]:
# Selected columns that have special 
recode_ordinal = ['ARBEIT', 'KBA05_ALTER1', 'KBA05_ALTER2', 'KBA05_ALTER3', 'KBA05_ALTER4', 'KBA05_ANHANG', 'KBA05_AUTOQUOT', 'KBA05_CCM1', 'KBA05_CCM2', 'KBA05_CCM3', 'KBA05_CCM4', 'KBA05_DIESEL', 'KBA05_FRAU', 'KBA05_HERST1', 'KBA05_HERST2', 'KBA05_HERST3', 'KBA05_HERST4', 'KBA05_HERST5', 'KBA05_KRSAQUOT', 'KBA05_KRSHERST1', 'KBA05_KRSHERST2', 'KBA05_KRSHERST3', 'KBA05_KRSKLEIN', 'KBA05_KRSOBER', 'KBA05_KRSVAN', 'KBA05_KRSZUL', 'KBA05_KW1', 'KBA05_KW2', 'KBA05_KW3', 'KBA05_MAXAH', 'KBA05_MAXBJ', 'KBA05_MAXSEG', 'KBA05_MAXVORB', 'KBA05_MOD1', 'KBA05_MOD2', 'KBA05_MOD3', 'KBA05_MOD4', 'KBA05_MOD8', 'KBA05_MOTOR', 'KBA05_MOTRAD', 'KBA05_SEG1', 'KBA05_SEG10', 'KBA05_SEG2', 'KBA05_SEG3', 'KBA05_SEG4', 'KBA05_SEG5', 'KBA05_SEG7', 'KBA05_SEG8', 'KBA05_SEG9', 'KBA05_VORB0', 'KBA05_VORB1', 'KBA05_VORB2', 'KBA05_ZUL1', 'KBA05_ZUL2', 'KBA05_ZUL3', 'KBA05_ZUL4', 'KOMBIALTER', 'RELAT_AB', 'ALTERSKATEGORIE_GROB']

In [ ]:
for var in recode_ordinal:
    l = customers[var].unique()
    print(var, sorted(l[~(np.isnan(l))]))

In [ ]:
# check values are ordered or not

def CheckOrdinal(data):
    need_recode = []
    var_ordinal = ['AKT_DAT_KL','ALTER_HH','ALTER_KIND1','ALTER_KIND2','ALTER_KIND3','ALTER_KIND4','ARBEIT','BALLRAUM','D19_BANKEN_ANZ_12','D19_BANKEN_ANZ_24','D19_BANKEN_DATUM','D19_BANKEN_OFFLINE_DATUM','D19_BANKEN_ONLINE_DATUM','D19_BANKEN_ONLINE_QUOTE_12','D19_GESAMT_ANZ_12','D19_GESAMT_ANZ_24','D19_GESAMT_DATUM','D19_GESAMT_OFFLINE_DATUM','D19_GESAMT_ONLINE_DATUM','D19_GESAMT_ONLINE_QUOTE_12','D19_TELKO_ANZ_12','D19_TELKO_ANZ_24','D19_TELKO_DATUM','D19_TELKO_OFFLINE_DATUM','D19_TELKO_ONLINE_DATUM','D19_TELKO_ONLINE_QUOTE_12','D19_VERSAND_ANZ_12','D19_VERSAND_ANZ_24','D19_VERSAND_DATUM','D19_VERSAND_OFFLINE_DATUM','D19_VERSAND_ONLINE_DATUM','D19_VERSAND_ONLINE_QUOTE_12','D19_VERSI_ANZ_12','D19_VERSI_ANZ_24','D19_VERSI_DATUM','D19_VERSI_OFFLINE_DATUM','D19_VERSI_ONLINE_DATUM','D19_VERSI_ONLINE_QUOTE_12','EWDICHTE','FINANZ_ANLEGER','FINANZ_HAUSBAUER','FINANZ_MINIMALIST','FINANZ_SPARER','FINANZ_UNAUFFAELLIGER','FINANZ_VORSORGER','FIRMENDICHTE','HH_EINKOMMEN_SCORE','INNENSTADT','KBA05_ALTER1','KBA05_ALTER2','KBA05_ALTER3','KBA05_ALTER4','KBA05_ANHANG','KBA05_ANTG1','KBA05_ANTG2','KBA05_ANTG3','KBA05_ANTG4','KBA05_AUTOQUOT','KBA05_BAUMAX','KBA05_CCM1','KBA05_CCM2','KBA05_CCM3','KBA05_CCM4','KBA05_DIESEL','KBA05_FRAU','KBA05_GBZ','KBA05_HERST1','KBA05_HERST2','KBA05_HERST3','KBA05_HERST4','KBA05_HERST5','KBA05_KRSAQUOT','KBA05_KRSHERST1','KBA05_KRSHERST2','KBA05_KRSHERST3','KBA05_KRSKLEIN','KBA05_KRSOBER','KBA05_KRSVAN','KBA05_KRSZUL','KBA05_KW1','KBA05_KW2','KBA05_KW3','KBA05_MAXAH','KBA05_MAXBJ','KBA05_MAXSEG','KBA05_MAXVORB','KBA05_MOD1','KBA05_MOD2','KBA05_MOD3','KBA05_MOD4','KBA05_MOD8','KBA05_MODTEMP','KBA05_MOTOR','KBA05_MOTRAD','KBA05_SEG1','KBA05_SEG10','KBA05_SEG2','KBA05_SEG3','KBA05_SEG4','KBA05_SEG5','KBA05_SEG7','KBA05_SEG8','KBA05_SEG9','KBA05_VORB0','KBA05_VORB1','KBA05_VORB2','KBA05_ZUL1','KBA05_ZUL2','KBA05_ZUL3','KBA05_ZUL4','KBA13_ALTERHALTER_30','KBA13_ALTERHALTER_45','KBA13_ALTERHALTER_60','KBA13_ALTERHALTER_61','KBA13_ANTG1','KBA13_ANTG2','KBA13_ANTG3','KBA13_ANTG4','KBA13_AUDI','KBA13_AUTOQUOTE','KBA13_BAUMAX','KBA13_BJ_1999','KBA13_BJ_2000','KBA13_BJ_2004','KBA13_BJ_2006','KBA13_BJ_2008','KBA13_BJ_2009','KBA13_BMW','KBA13_CCM_0_1400','KBA13_CCM_1000','KBA13_CCM_1200','KBA13_CCM_1400','KBA13_CCM_1401_2500','KBA13_CCM_1500','KBA13_CCM_1600','KBA13_CCM_1800','KBA13_CCM_2000','KBA13_CCM_2500','KBA13_CCM_2501','KBA13_CCM_3000','KBA13_CCM_3001','KBA13_FAB_ASIEN','KBA13_FAB_SONSTIGE','KBA13_FIAT','KBA13_FORD','KBA13_GBZ','KBA13_HALTER_20','KBA13_HALTER_25','KBA13_HALTER_30','KBA13_HALTER_35','KBA13_HALTER_40','KBA13_HALTER_45','KBA13_HALTER_50','KBA13_HALTER_55','KBA13_HALTER_60','KBA13_HALTER_65','KBA13_HALTER_66','KBA13_HERST_ASIEN','KBA13_HERST_AUDI_VW','KBA13_HERST_BMW_BENZ','KBA13_HERST_EUROPA','KBA13_HERST_FORD_OPEL','KBA13_HERST_SONST','KBA13_HHZ','KBA13_KMH_0_140','KBA13_KMH_110','KBA13_KMH_140','KBA13_KMH_140_210','KBA13_KMH_180','KBA13_KMH_210','KBA13_KMH_211','KBA13_KMH_250','KBA13_KMH_251','KBA13_KRSAQUOT','KBA13_KRSHERST_AUDI_VW','KBA13_KRSHERST_BMW_BENZ','KBA13_KRSHERST_FORD_OPEL','KBA13_KRSSEG_KLEIN','KBA13_KRSSEG_OBER','KBA13_KRSSEG_VAN','KBA13_KRSZUL_NEU','KBA13_KW_0_60','KBA13_KW_110','KBA13_KW_120','KBA13_KW_121','KBA13_KW_30','KBA13_KW_40','KBA13_KW_50','KBA13_KW_60','KBA13_KW_61_120','KBA13_KW_70','KBA13_KW_80','KBA13_KW_90','KBA13_MAZDA','KBA13_MERCEDES','KBA13_MOTOR','KBA13_NISSAN','KBA13_OPEL','KBA13_PEUGEOT','KBA13_RENAULT','KBA13_SEG_GELAENDEWAGEN','KBA13_SEG_GROSSRAUMVANS','KBA13_SEG_KLEINST','KBA13_SEG_KLEINWAGEN','KBA13_SEG_KOMPAKTKLASSE','KBA13_SEG_MINIVANS','KBA13_SEG_MINIWAGEN','KBA13_SEG_MITTELKLASSE','KBA13_SEG_OBEREMITTELKLASSE','KBA13_SEG_OBERKLASSE','KBA13_SEG_SONSTIGE','KBA13_SEG_SPORTWAGEN','KBA13_SEG_UTILITIES','KBA13_SEG_VAN','KBA13_SEG_WOHNMOBILE','KBA13_SITZE_4','KBA13_SITZE_5','KBA13_SITZE_6','KBA13_TOYOTA','KBA13_VORB_0','KBA13_VORB_1','KBA13_VORB_1_2','KBA13_VORB_2','KBA13_VORB_3','KBA13_VW','KKK','KOMBIALTER','MOBI_RASTER','MOBI_REGIO','ORTSGR_KLS9','PLZ8_ANTG1','PLZ8_ANTG2','PLZ8_ANTG3','PLZ8_ANTG4','PLZ8_BAUMAX','PLZ8_GBZ','PLZ8_HHZ','RELAT_AB','RT_KEIN_ANREIZ','RT_SCHNAEPPCHEN','RT_UEBERGROESSE','SEMIO_DOM','SEMIO_ERL','SEMIO_FAM','SEMIO_KAEM','SEMIO_KRIT','SEMIO_KULT','SEMIO_LUST','SEMIO_MAT','SEMIO_PFLICHT','SEMIO_RAT','SEMIO_REL','SEMIO_SOZ','SEMIO_TRADV','SEMIO_VERT','STRUKTURTYP','UMFELD_ALT','UMFELD_JUNG','VHA','VHN','VK_DHT4A','VK_DISTANZ','VK_ZG11','W_KEIT_KIND_HH','WOHNDAUER_2008','ALTERSKATEGORIE_GROB']
    for var in var_ordinal:
        l = data[var].unique()
        w = l[~(np.isnan(l))]
        end = len(w) -1
        if w.max() - w.min() + 1 != len(w) and sorted(w)[end] - sorted(w)[end-1] != 1:
            print(var, sorted(w))
            need_recode.append(var)
    print('All checked!') 

In [ ]:
CheckOrdinal(customers)

In [ ]:
CheckOrdinal(azdias)

In [ ]:
## recode 9 into missing value

def RecoderOrdinal(data):
    recode_ordinal = ['ARBEIT', 'KBA05_ALTER1', 'KBA05_ALTER2', 'KBA05_ALTER3', 'KBA05_ALTER4', 'KBA05_ANHANG', 'KBA05_AUTOQUOT', 'KBA05_CCM1', 'KBA05_CCM2', 'KBA05_CCM3', 'KBA05_CCM4', 'KBA05_DIESEL', 'KBA05_FRAU', 'KBA05_HERST1', 'KBA05_HERST2', 'KBA05_HERST3', 'KBA05_HERST4', 'KBA05_HERST5', 'KBA05_KRSAQUOT', 'KBA05_KRSHERST1', 'KBA05_KRSHERST2', 'KBA05_KRSHERST3', 'KBA05_KRSKLEIN', 'KBA05_KRSOBER', 'KBA05_KRSVAN', 'KBA05_KRSZUL', 'KBA05_KW1', 'KBA05_KW2', 'KBA05_KW3', 'KBA05_MAXAH', 'KBA05_MAXBJ', 'KBA05_MAXSEG', 'KBA05_MAXVORB', 'KBA05_MOD1', 'KBA05_MOD2', 'KBA05_MOD3', 'KBA05_MOD4', 'KBA05_MOD8', 'KBA05_MOTOR', 'KBA05_MOTRAD', 'KBA05_SEG1', 'KBA05_SEG10', 'KBA05_SEG2', 'KBA05_SEG3', 'KBA05_SEG4', 'KBA05_SEG5', 'KBA05_SEG7', 'KBA05_SEG8', 'KBA05_SEG9', 'KBA05_VORB0', 'KBA05_VORB1', 'KBA05_VORB2', 'KBA05_ZUL1', 'KBA05_ZUL2', 'KBA05_ZUL3', 'KBA05_ZUL4', 'KOMBIALTER', 'RELAT_AB', 'ALTERSKATEGORIE_GROB']
    for var in recode_ordinal:
        if 9 in data[var].unique():
            print(var, 'has been recoded.')
            data.loc[:,var] = np.where(data[var]==9, np.nan, data[var])
        
    print('All chceked!')

In [ ]:
RecoderOrdinal(customers)

In [ ]:
RecoderOrdinal(azdias)

In [ ]:
CheckOrdinal(customers)

In [ ]:
CheckOrdinal(azdias)

#### Data Preprocessing for variables needed calculating

In [ ]:
for var in var_date:
    print(var, customers[var].nunique())

In [ ]:
def RecoderDatetime(data):
    var_date = ['EINGEFUEGT_AM','EINGEZOGENAM_HH_JAHR','GEBURTSJAHR','MIN_GEBAEUDEJAHR']
    t1 = pd.to_datetime('2020-1-1')
    for var in var_date:
        data.loc[:,var] = data[var].astype('datetime64[ns]')
        data.loc[:,var] = t1 - data[var]

In [ ]:
%%time
RecoderDatetime(customers)

In [ ]:
%%time
RecoderDatetime(azdias)

#### Data Preprocessing for categorical variables

In [ ]:
for var in var_categorical:
    print(var, customers[var].nunique())

In [ ]:
for var in var_categorical:
    if customers[var].nunique() < 50:
        print(var, customers[var].unique())

In [ ]:
customers.to_csv('customers.csv')

In [ ]:
azdias.to_csv('azdias.csv')

In [ ]:
def CheckCategories(data1, data2):
    var_categorical = ['AGER_TYP','ALTERSKATEGORIE_FEIN','CAMEO_DEU_2015','CAMEO_DEUG_2015','CAMEO_INTL_2015','CJT_GESAMTTYP','CJT_KATALOGNUTZER','CJT_TYP_1','CJT_TYP_2','CJT_TYP_3','CJT_TYP_4','CJT_TYP_5','CJT_TYP_6','D19_BANKEN_DIREKT','D19_BANKEN_GROSS','D19_BANKEN_LOKAL','D19_BANKEN_REST','D19_BEKLEIDUNG_GEH','D19_BEKLEIDUNG_REST','D19_BILDUNG','D19_BIO_OEKO','D19_BUCH_CD','D19_DIGIT_SERV','D19_DROGERIEARTIKEL','D19_ENERGIE','D19_FREIZEIT','D19_GARTEN','D19_HANDWERK','D19_HAUS_DEKO','D19_KINDERARTIKEL','D19_KONSUMTYP','D19_KONSUMTYP_MAX','D19_KOSMETIK','D19_LEBENSMITTEL','D19_LETZTER_KAUF_BRANCHE','D19_LOTTO','D19_NAHRUNGSERGAENZUNG','D19_RATGEBER','D19_REISEN','D19_SAMMELARTIKEL','D19_SCHUHE','D19_SONSTIGE','D19_SOZIALES','D19_TECHNIK','D19_TELKO_MOBILE','D19_TELKO_REST','D19_TIERARTIKEL','D19_VERSAND_REST','D19_VERSICHERUNGEN','D19_VOLLSORTIMENT','D19_WEIN_FEINKOST','DSL_FLAG','EXTSEL992','FINANZTYP','GEBAEUDETYP','GEBAEUDETYP_RASTER','GEMEINDETYP','GFK_URLAUBERTYP','HEALTH_TYP','KBA05_HERSTTEMP','KBA05_MAXHERST','KK_KUNDENTYP','KONSUMNAEHE','LP_FAMILIE_FEIN','LP_FAMILIE_GROB','LP_LEBENSPHASE_FEIN','LP_LEBENSPHASE_GROB','LP_STATUS_FEIN','LP_STATUS_GROB','NATIONALITAET_KZ','ONLINE_AFFINITAET','OST_WEST_KZ','PRAEGENDE_JUGENDJAHRE','REGIOTYP','RETOURTYP_BK_S','SHOPPER_TYP','TITEL_KZ','VERDICHTUNGSRAUM','VERS_TYP','WOHNLAGE','ZABEOTYP']
    for var in var_categorical:
        if set(data1[var].unique()) > set(data2[var].unique()):
            print(var, 'has different categories.')
        else:
            print(var, 'is checked.')

In [ ]:
CheckCategories(data1=customers, data2=azdias)

In [ ]:
# feature encoding for categorical variables

def RecoderCatergorical(data1, data2):
    total_cnt = data1.shape[0]
    var_categorical = ['AGER_TYP','ALTERSKATEGORIE_FEIN','CAMEO_DEU_2015','CAMEO_DEUG_2015','CAMEO_INTL_2015','CJT_GESAMTTYP','CJT_KATALOGNUTZER','CJT_TYP_1','CJT_TYP_2','CJT_TYP_3','CJT_TYP_4','CJT_TYP_5','CJT_TYP_6','D19_BANKEN_DIREKT','D19_BANKEN_GROSS','D19_BANKEN_LOKAL','D19_BANKEN_REST','D19_BEKLEIDUNG_GEH','D19_BEKLEIDUNG_REST','D19_BILDUNG','D19_BIO_OEKO','D19_BUCH_CD','D19_DIGIT_SERV','D19_DROGERIEARTIKEL','D19_ENERGIE','D19_FREIZEIT','D19_GARTEN','D19_HANDWERK','D19_HAUS_DEKO','D19_KINDERARTIKEL','D19_KONSUMTYP','D19_KONSUMTYP_MAX','D19_KOSMETIK','D19_LEBENSMITTEL','D19_LETZTER_KAUF_BRANCHE','D19_LOTTO','D19_NAHRUNGSERGAENZUNG','D19_RATGEBER','D19_REISEN','D19_SAMMELARTIKEL','D19_SCHUHE','D19_SONSTIGE','D19_SOZIALES','D19_TECHNIK','D19_TELKO_MOBILE','D19_TELKO_REST','D19_TIERARTIKEL','D19_VERSAND_REST','D19_VERSICHERUNGEN','D19_VOLLSORTIMENT','D19_WEIN_FEINKOST','DSL_FLAG','EXTSEL992','FINANZTYP','GEBAEUDETYP','GEBAEUDETYP_RASTER','GEMEINDETYP','GFK_URLAUBERTYP','HEALTH_TYP','KBA05_HERSTTEMP','KBA05_MAXHERST','KK_KUNDENTYP','KONSUMNAEHE','LP_FAMILIE_FEIN','LP_FAMILIE_GROB','LP_LEBENSPHASE_FEIN','LP_LEBENSPHASE_GROB','LP_STATUS_FEIN','LP_STATUS_GROB','NATIONALITAET_KZ','ONLINE_AFFINITAET','OST_WEST_KZ','PRAEGENDE_JUGENDJAHRE','REGIOTYP','RETOURTYP_BK_S','SHOPPER_TYP','TITEL_KZ','VERDICHTUNGSRAUM','VERS_TYP','WOHNLAGE','ZABEOTYP']
    for var in var_categorical:
        tmp_dict = (data1[var].value_counts(dropna=False) / total_cnt).to_dict()
        data1.loc[:,var] = data1[var].map(tmp_dict)
        data2.loc[:,var] = data2[var].map(tmp_dict)


In [ ]:
FrequencyEncoding(data1=customers, data2=azdias)

#### Missing value imputation

* impute zero for ordinal variables and binary variables
* impute median value for numeric variables and datetime variables
* we have already taken missing value into account when performing frequency encoding, so we do not need to impute missing value for categorical variables

In [ ]:
def CheckMissing(data):
    total_cnt = data.shape[0]
    for var in var_numeric+var_binary+var_ordinal+var_date:
        print(var, round(data[var].isnull().sum() / total_cnt,4))

In [ ]:
CheckMissing(customers)

In [ ]:
CheckMissing(azdias)

In [ ]:
def MissingImpute(data):
    data.loc[:,var_numeric] = data[var_numeric].fillna(data[var_numeric].median())
    data.loc[:,var_date] = data[var_date].fillna(data[var_date].median())
    data.loc[:,var_binary] = data[var_binary].fillna(0)
    data.loc[:,var_ordinal] = data[var_ordinal].fillna(0)

#### Drop for redundent columns

In [ ]:
# fine vs rough

In [ ]:
#Custom Transformer that extracts columns passed as argument to its constructor 
class FeatureSelector( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        return X[ self._feature_names ] 

In [ ]:
class BinaryTransformer(BaseEstimator, TransformerMixin):
    #Class Constructor
    def __init__( self ):
        pass
        
    #Return self, nothing else to do here
    def fit( self, X, y = None ):
        return self 
    
    #Custom transform method we wrote that creates aformentioned features and drops redundant ones 
    def transform(self, X, y = None):
        X.loc[:,'ANREDE_KZ'] = X['ANREDE_KZ'].map({1:0,2:1})
        X = X.map({0:0,1:1})
        
        return X.values

In [ ]:
class OrdinalTransformer(BaseEstimator, TransformerMixin):
    #Class Constructor
    def __init__( self ):
        pass
        
    #Return self, nothing else to do here
    def fit( self, X, y = None ):
        return self 
    
    #Custom transform method we wrote that creates aformentioned features and drops redundant ones 
    def transform(self, X, y = None):
        X.loc[:,'ANREDE_KZ'] = X['ANREDE_KZ'].map({1:0,2:1})
        X = X.map({0:0,1:1})
        
        return X.values

In [ ]:
## recode 9 into missing value

def RecoderOrdinal(data):
    recode_ordinal = ['ARBEIT', 'KBA05_ALTER1', 'KBA05_ALTER2', 'KBA05_ALTER3', 'KBA05_ALTER4', 'KBA05_ANHANG', 'KBA05_AUTOQUOT', 'KBA05_CCM1', 'KBA05_CCM2', 'KBA05_CCM3', 'KBA05_CCM4', 'KBA05_DIESEL', 'KBA05_FRAU', 'KBA05_HERST1', 'KBA05_HERST2', 'KBA05_HERST3', 'KBA05_HERST4', 'KBA05_HERST5', 'KBA05_KRSAQUOT', 'KBA05_KRSHERST1', 'KBA05_KRSHERST2', 'KBA05_KRSHERST3', 'KBA05_KRSKLEIN', 'KBA05_KRSOBER', 'KBA05_KRSVAN', 'KBA05_KRSZUL', 'KBA05_KW1', 'KBA05_KW2', 'KBA05_KW3', 'KBA05_MAXAH', 'KBA05_MAXBJ', 'KBA05_MAXSEG', 'KBA05_MAXVORB', 'KBA05_MOD1', 'KBA05_MOD2', 'KBA05_MOD3', 'KBA05_MOD4', 'KBA05_MOD8', 'KBA05_MOTOR', 'KBA05_MOTRAD', 'KBA05_SEG1', 'KBA05_SEG10', 'KBA05_SEG2', 'KBA05_SEG3', 'KBA05_SEG4', 'KBA05_SEG5', 'KBA05_SEG7', 'KBA05_SEG8', 'KBA05_SEG9', 'KBA05_VORB0', 'KBA05_VORB1', 'KBA05_VORB2', 'KBA05_ZUL1', 'KBA05_ZUL2', 'KBA05_ZUL3', 'KBA05_ZUL4', 'KOMBIALTER', 'RELAT_AB', 'ALTERSKATEGORIE_GROB']
    for var in recode_ordinal:
        if 9 in data[var].unique():
            print(var, 'has been recoded.')
            data.loc[:,var] = np.where(data[var]==9, np.nan, data[var])
        
    print('All chceked!')

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')